In [1]:
import random

import numpy as np
import pandas as pd

import torch

torch.cuda.empty_cache()

def seed_all(seed: int) -> None:
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    random.seed(seed)

In [2]:
# Фиксируем random seed
SEED = 42
seed_all(SEED)

In [3]:
from pathlib import Path

DATA_PATH = Path('../data/')
DATA_PATH.mkdir(parents=True, exist_ok=True)

DATA_CACHE = DATA_PATH / Path('cache_dir/')
DATA_CACHE.mkdir(parents=True, exist_ok=True)

import pandas as pd

pd.set_option('display.max_colwidth', 500) 

In [4]:
import sys
import os

project_path = os.path.abspath(os.path.join(os.getcwd(), "../.."))
sys.path.append(project_path)

In [5]:
from app.models.translate_model import TranslationModel, TranslationModelQAT

In [9]:
from pathlib import Path
import json

long_text_path = Path("long_text.json")

with open(long_text_path, "r", encoding="utf-8") as f:
    data = json.load(f)

texts = data["texts"]
source_lang = data["source_lang"]
target_langs = data["target_langs"]
data

{'texts': ['Глава 1. Призыв к битве\n\nМир древности был полон чудес и чудовищ. В те дни на земле великих правителей и легендарных героев, сражения не ограничивались только оружием и армиями. Они были испытаниями силы духа, мужества и ума. Одним из таких испытаний стали гонки на колесницах, что стало невероятно популярным в различных уголках мира, и даже в Иудее, где собрались великие герои Библии.\n\nЭти гонки, однако, не были простыми. Они не были лишь забава ради. Каждая колесница олицетворяла не только военную мощь, но и внутренний бой, борьбу с судьбой, и даже божественные вмешательства. Колесницы, запряженные сильными животными, мчались по древним трассам, где каждое движение было решающим. Гонщики, ставшие участниками этих состязаний, были героями, чьи имена известны всем — Моисей, Давид, Илия и многие другие.\n\nГлава 2. Моисей и его колесница\n\nМоисей, великий вождь и пророк, был на старости лет, но в его глазах горел огонь непокорного духа. Он решил принять участие в этих го

## Квантованная модель

In [10]:
model = TranslationModelQAT(
    model_name="michaelfeil/ct2fast-m2m100_418M",
    # model_name="michaelfeil/ct2fast-m2m100_1.2B",
    cache_dir=DATA_CACHE,
    device="cpu"
)

In [11]:
# Переводим тексты
translations = model.translate_batch(
    texts=texts,
    source_lang=source_lang,
    target_langs=target_langs
)

# Выводим результат
for lang, translated_list in translations.items():
    print(f"Переводы на язык '{lang}':")
    for idx, translated_text in enumerate(translated_list):
        print(f"  [{idx + 1}] {translated_text}")
    print()

Переводы на язык 'uk':
  [1] Розділ 1. Заклик до битви Світ давньості був наповнений чудесами і монстрами. У ті дні на землі великих правителів і легендарних героїв, битви не обмежувалися лише зброєю і армією. Вони були випробуваннями сили духу, мужності і розуму. Одним з таких випробувань стали гонки на колесницях, що стало неймовірно популярним в різних куточках світу, і навіть в Євреї, де зібралися великі герої Біблії. Однак ці гонки не були простими. Вони були не тільки веселою. Кожна колесниця олицтувала не тільки військову силу, але і внутрішній бій, боротьбу з долею, і навіть божественні втручання. Колесниці, запліднені сильними тваринами, м'ячали по стародавніх трасах, де кожен рух був вирішальним. Гонщики, які стали учасниками цих змагань, були героями, чиї імена відомі всім – Мойсею, Давиду, Іллію та багатьом іншим. Розділ 2. Мойсей і його колесниця Мойсей, великий вождь і пророк, був на старості років, але в його очах горів вогонь непокорного духу. Він вирішив взяти участь у

## Обычная модель

In [ ]:
model = TranslationModel(
    model_name="facebook/m2m100_418M",
    # model_name="facebook/m2m100_1.2B",
    cache_dir=DATA_CACHE,
    device="cpu"
)

In [10]:
# Переводим тексты
translations = model.translate_batch(
    texts=texts,
    source_lang=source_lang,
    target_langs=target_langs
)

# Выводим результат
for lang, translated_list in translations.items():
    print(f"Переводы на язык '{lang}':")
    for idx, translated_text in enumerate(translated_list):
        print(f"  [{idx + 1}] {translated_text}")
    print()

d:\projects\env_ml\.venv\Lib\site-packages\transformers\generation\configuration_utils.py:634: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Переводы на язык 'en':
  [1] Chapter 1 The call to the battle The world of ancient times was full of miracles and monsters. In those days on the ground of great rulers and legendary heroes, the battles were not limited to weapons and armies. They were the trials of the strength of spirit, courage and mind. One of these trials was the races on the wheels, which became incredibly popular in different corners of the world, and even in Judea, where the great heroes of the Bible gathered. However, these races were not easy. They were not just fun. Each wheel represented not only the military power, but also the internal battle, the fate struggle, and even the divine interventions. The wheels, stretched by strong animals, walked on ancient trails, where every movement was decisive. The runners who participated in these competitions were heroes whose names are known to everyone—Moses, David, Elijah and many others. Chapter 2 . Moses and his wardrobe Moses, the great leader and prophet, were o